## Preambule

In [1]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

In [2]:
import json
f = open('X:/user/dekkerm/Data/Maps/Somaliland.json', encoding='utf8')
somaliland_data = json.load(f)

## Paths

In [3]:
path_data = Path("K:/Data/Data_effortsharing/DataUpdate_ongoing/startyear_2021/")
path_data_gen = Path("K:/Data/Data_effortsharing/DataUpdate_ongoing/")
path_figs = Path("K:/Code/effort-sharing/") / "Figures" / "Paper_Robiou2023"

## Read data files

In [4]:
xr_dataread = xr.open_dataset(path_data / "xr_dataread.nc")
xr_combs = xr.open_dataset(path_data / "xr_comb_GHG_excl.nc")
xr_temps = xr.open_dataset(path_data / "xr_combtemps_GHG_excl.nc")
all_regions_iso = np.load(path_data_gen / "all_regions.npy")
all_regions_names = np.array(list(np.load(path_data_gen / "all_regions_names.npy")[:-1])+['Earth']) # Only in old xr_dataread version, should be updated when reran
all_countries_iso = np.load(path_data_gen / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data_gen / "all_countries_names.npy", allow_pickle=True)
only_regions_iso = ['CVF', 'G20', "EU", "G7", "NA", "AU", "AF", "SIDS", "LDC", "WORLD"]
only_regions_names = ['Climate Vulnerability Forum','G20',"European Union","G7","Northern America","Australasia","African Group","Small Island Developing States","Least Developed Countries", 'World']

In [5]:
df_g = pd.read_excel(Path("X:/user/dekkerm/Data/") / "UNFCCC_Parties_Groups_noeu.xlsx", sheet_name = "Country groups")
countries_iso = np.array(df_g["Country ISO Code"])
countries_name = np.array(df_g["Name"])
countries_iso = np.array(df_g["Country ISO Code"])
group_cvf = countries_iso[np.array(df_g["CVF (24/10/22)"]) == 1]
group_g20 = countries_iso[np.array(df_g["G20"]) == 1]
group_eu = countries_iso[np.array(df_g["EU"]) == 1]
group_g7 = countries_iso[np.array(df_g["G7"]) == 1]
group_na = countries_iso[np.array(df_g["Northern America"]) == 1]
group_um = countries_iso[np.array(df_g["Umbrella"]) == 1]
group_au = countries_iso[np.array(df_g["Australasia"]) == 1]
group_af = countries_iso[np.array(df_g["African Group"]) == 1]
group_sids = countries_iso[np.array(df_g["SIDS"]) == 1]
group_ldc = countries_iso[np.array(df_g["LDC"]) == 1]
group_eg = countries_iso[np.array(df_g["European Group"]) == 1]
group_world = np.copy(countries_iso)

In [6]:
temps_1 = xr_temps.ECPC.sel(Time=2030, Discount_factor=0, Historical_startyear=1990, Risk=0.5, Scenario='SSP2').mean(dim=['Conditionality', 'Ambition'])
#temps_2 = xr_temps.Approach2.sel(Time=2030, Discount_factor=0, Historical_startyear=1990, Risk=0.5, Scenario='SSP2').mean(dim=['Conditionality', 'Ambition'])

## Plot

In [7]:
fig = make_subplots(rows=1, cols=1,
                    specs=[[{'type': 'choropleth'}]],
                    horizontal_spacing = 0., vertical_spacing=0.0)

for i in range(1):
    dath = [temps_1][i]
    dat = np.array(dath)
    scale = 'RdYlGn_r'
    dc = dict(orientation='h', x=0.35, y=0.0, xanchor='left', yanchor='bottom', title='<b>Climate target', tickvals=[1.0, 1.5, 2.0, 2.5, 3, 3.5, 4],
                        ticktext=['1.0  &deg;C', '1.5  &deg;C', '2.0 &deg;C', '2.5 &deg;C', '3.0 &deg;C', '3.5 &deg;C', '4.0 &deg;C'],
                        tickfont=dict(size=14),
                        len=0.4, lenmode='fraction', thickness=20, thicknessmode='pixels', titlefont=dict(size=14))
    #Grey out nans
    fig.add_trace(go.Choropleth(
            locations=['ATA'],
            z = [0.5],
            locationmode = 'ISO-3',
            colorscale ='Greys',
            zmax = 4.5,
            zmin = 0.5,
            text = ['hoi'],
            hovertemplate  = '%{text}',
            name="",
            marker_line_color='black', 
            marker_line_width=0,
            showscale=False,
        ), [1, 2][i], 1)
    
    # Make somaliland equal to SOM
    data = data = [['SOM',float(dath.sel(Region='SOM'))],['SOL',float(dath.sel(Region='SOM'))]]
    df = pd.DataFrame(data,columns=['sov_a3','population'])
    
    fig.add_trace(go.Choropleth(
            locations=np.array((dath).Region),
            z = dat,
            locationmode = 'ISO-3',
            colorscale = scale,
                zmax = 4.,
                zmin = 1.0,
            text = [str(r)+": "+str(np.round(dat[r_i], 1))+" &deg;C" for r_i, r in enumerate(np.array((dath).Region))],
            hovertemplate  = '%{text}',
            name="",
            colorbar=dc,
            marker_line_color='black', 
            marker_line_width=1,
            showscale=[True, False][i],
        ), [1, 2][i], 1)
    
    
    fig.add_trace(
        go.Choropleth(
        locations=group_eu,
        z = np.array([dat[np.where(all_regions_iso == "EU")[0][0]]]*len(group_eu)),
        locationmode = 'ISO-3',
        colorscale = scale,
            zmax = 4.,
            zmin = 1.0,
        text = [str(r)+": "+str(np.round(dat[np.where(all_regions_iso == "EU")[0][0]], 1))+" &deg;C" for r_i, r in enumerate(group_eu)],
        hovertemplate  = '%{text}',
        name="",
        colorbar=dc,
        marker_line_color='black', 
        marker_line_width=1,
        showscale=[True, False][i],
        ), [1, 2][i], 1)

fs=20
fig['layout'].update(
    annotations=[dict(x=0.5, y=1.05, text='<b>NDC alignment of an equal per capita approach</b>', xref='paper', yref='paper', showarrow=False, font=dict(color='black', size=fs)),
                 #dict(x=0.15, y=0.52, text='<b>(b) NDC alignment of approach 2', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs)),
                 ])

fig.update_geos(visible=True,
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showframe=False,
                resolution=110,
                landcolor="lightgrey",
                showcoastlines=False,)
fig.update_traces(colorbar=dict(
    title=dict(
        side="bottom"
    )
))

fig.update_layout(height=800, width=1450)
fig.write_image(path_figs / "Figure_5.png", scale=5)
fig.show()